In [3]:
import pandas as pd
import numpy as np
import string  
import nltk
import re
#import scipy
#import matplotlib.pyplot as plt
#import urllib 
import sklearn
from sklearn import preprocessing
#from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error, f1_score

!pip install catboost 
from catboost import CatBoostRegressor, CatBoostClassifier
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

display(train)
display(test)

  Using cached plotly-4.9.0-py2.py3-none-any.whl (12.9 MB)
  Using cached graphviz-0.14.1-py2.py3-none-any.whl (18 kB)
Processing c:\users\hijos.asus\appdata\local\pip\cache\wheels\f9\8d\8d\f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf\retrying-1.3.3-py3-none-any.whl


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [4]:
#Rellenando nulos
train['keyword'].fillna('no keyword', inplace = True) 
train['keyword'] = train['keyword'].str.replace('%20', ' ')
train['location'].fillna('no location', inplace = True)

test['keyword'].fillna('no keyword', inplace = True) 
test['keyword'] = test['keyword'].str.replace('%20', ' ')
test['location'].fillna('no location', inplace = True)

train['length'] = train['text'].str.len()
test['length'] = test['text'].str.len()

train.head()

,id,keyword,location,text,target,length
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88


In [5]:
#limpiando el texto

import nltk
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

#FUNCIONES UTILES

def only_letters(tweet):
    tweet = re.sub(r'http\S*', '', tweet)
    tweet = re.sub(r'[^a-z\s]', '', tweet)
    return tweet

stop_words=set(stopwords.words("english"))

def filter_stopwords(tokenized_text):
    filtered_words=[]
    for w in tokenized_text:
        if w not in stop_words:
            filtered_words.append(w)
    return filtered_words

lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(tweet):
    lemmatized_words = []
    for word in tweet:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    return lemmatized_words

def transform_to_text(tweet_words):
    return " ".join(tweet_words)

#Cleaning text

train['clean_text'] = train['text'].str.lower()

train['clean_text'] = train['clean_text'].apply(only_letters)    

test['clean_text'] = test['text'].str.lower()

test['clean_text'] = test['clean_text'].apply(only_letters) 

#Tokenización

train['clean_text'] = train['clean_text'].apply(word_tokenize)

test['clean_text'] = test['clean_text'].apply(word_tokenize)

#Remove stopwords

train['clean_text'] = train['clean_text'].apply(filter_stopwords) 

test['clean_text'] = test['clean_text'].apply(filter_stopwords) 


#Lemmatization                                                                       

train['clean_text'] = train['clean_text'].apply(lemmatize_tweet)  

train['clean_text'] = train['clean_text'].apply(transform_to_text)

test['clean_text'] = test['clean_text'].apply(lemmatize_tweet)  

test['clean_text'] = test['clean_text'].apply(transform_to_text)

display(train)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hijos.ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id,keyword,location,text,target,length,clean_text
0,1,no keyword,no location,Our Deeds are the Reason of this #earthquake M...,1,69,deed reason earthquake may allah forgive u
1,4,no keyword,no location,Forest fire near La Ronge Sask. Canada,1,38,forest fire near la ronge sask canada
2,5,no keyword,no location,All residents asked to 'shelter in place' are ...,1,133,resident asked shelter place notified officer ...
3,6,no keyword,no location,"13,000 people receive #wildfires evacuation or...",1,65,people receive wildfire evacuation order calif...
4,7,no keyword,no location,Just got sent this photo from Ruby #Alaska as ...,1,88,got sent photo ruby alaska smoke wildfire pour...
...,...,...,...,...,...,...,...
7608,10869,no keyword,no location,Two giant cranes holding a bridge collapse int...,1,83,two giant crane holding bridge collapse nearby...
7609,10870,no keyword,no location,@aria_ahrary @TheTawniest The out of control w...,1,125,ariaahrary thetawniest control wild fire calif...
7610,10871,no keyword,no location,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,65,utckm volcano hawaii
7611,10872,no keyword,no location,Police investigating after an e-bike collided ...,1,137,police investigating ebike collided car little...


In [6]:
#agregando algunos features

keywords_list = list(set(train.keyword))  #todas las keywords únicas

locations_list = list(set(train.location)) #todas las locaciones únicas
locations_list.remove('no location')

def amount_keywords(tweet):
    cont = 0
    for keyword in keywords_list:
        if keyword in tweet:
            cont += 1
    return cont

train['amount_words'] = train['clean_text'].str.split(' ').apply(lambda x: len(x))  

test['amount_words'] = test['clean_text'].str.split(' ').apply(lambda x: len(x))  

train['amount_keywords_in_tweet'] = train['clean_text'].apply(lambda x: amount_keywords(x))

test['amount_keywords_in_tweet'] = test['clean_text'].apply(lambda x: amount_keywords(x))

def amount_locations(tweet):    #esta función podría unificarse con la de keywords pasandole por parámetro la lista a recorrer
    cont = 0
    for location in locations_list:
        if location in tweet:
            cont += 1
    return cont

train['amount_locations_in_tweet'] = train['clean_text'].apply(lambda x: amount_locations(x))

test['amount_locations_in_tweet'] = test['clean_text'].apply(lambda x: amount_locations(x))

In [7]:
#más features
def get_hashtags(s):
    return list(part[1:] for part in s.split() if part.startswith('#'))

train['hashtags'] = train['text'].apply(get_hashtags)
test['hashtags'] = test['text'].apply(get_hashtags)

def hashtag_length_proportion(hashtags, length):
    return len(''.join(hashtags))/length

train['len_hash_over_text'] = train.apply(lambda data: hashtag_length_proportion(data['hashtags'], data['length']), axis=1)
test['len_hash_over_text'] = test.apply(lambda data: hashtag_length_proportion(data['hashtags'], data['length']), axis=1)

def get_mentioned_users(s):
    return list(part[1:] for part in s.split() if part.startswith('@'))

train['users'] = train['text'].apply(get_mentioned_users)
test['users'] = test['text'].apply(get_mentioned_users)

def amount_hashtags(l):
    return len(l)

train['amount_hashtags'] = train['hashtags'].apply(amount_hashtags)
test['amount_hashtags'] = test['hashtags'].apply(amount_hashtags)

def amount_users(l):
    return len(l)

def has_url(text):
    return int('http' in text)

train['has_url'] = train['text'].apply(has_url)
test['has_url'] = test['text'].apply(has_url)

def find_url(text):
    urls = re.findall(r'(https?://\S+)', text)
    return urls

train['urls'] = train['text'].apply(find_url)
test['urls'] = test['text'].apply(find_url)
train['amount_urls'] = train['urls'].apply(lambda x: len(x))
test['amount_urls'] = test['urls'].apply(lambda x: len(x))

train['amount_users'] = train['users'].apply(amount_users)
test['amount_users'] = test['users'].apply(amount_users)


In [8]:
#construyendo los datos de entrenamiento y de test
selected_df = train[['length','amount_words','amount_keywords_in_tweet','amount_locations_in_tweet', 'amount_urls','amount_users', 'amount_hashtags','len_hash_over_text', 'has_url']] 

X, y = selected_df,train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [17]:
#Pruebo primero con CatBoostRegressor
#CatBoostRegressor por defecto
model = CatBoostRegressor(random_seed = 41)
model.fit(X_train, y_train)
preds = model.predict(X_test)

Learning rate set to 0.053835
0:	learn: 0.4908195	total: 4.38ms	remaining: 4.37s
1:	learn: 0.4868907	total: 8.46ms	remaining: 4.22s
2:	learn: 0.4835794	total: 12.6ms	remaining: 4.18s
3:	learn: 0.4806080	total: 16.7ms	remaining: 4.16s
4:	learn: 0.4780186	total: 20.7ms	remaining: 4.12s
5:	learn: 0.4755176	total: 24ms	remaining: 3.98s
6:	learn: 0.4731890	total: 27.4ms	remaining: 3.89s
7:	learn: 0.4708731	total: 31.7ms	remaining: 3.93s
8:	learn: 0.4690652	total: 34.6ms	remaining: 3.81s
9:	learn: 0.4672509	total: 38.7ms	remaining: 3.83s
10:	learn: 0.4656755	total: 42.7ms	remaining: 3.84s
11:	learn: 0.4640961	total: 46.7ms	remaining: 3.85s
12:	learn: 0.4626838	total: 50.8ms	remaining: 3.86s
13:	learn: 0.4614090	total: 54.9ms	remaining: 3.87s
14:	learn: 0.4600285	total: 59.1ms	remaining: 3.88s
15:	learn: 0.4587520	total: 63.1ms	remaining: 3.88s
16:	learn: 0.4576459	total: 67.3ms	remaining: 3.89s
17:	learn: 0.4565588	total: 71.4ms	remaining: 3.9s
18:	learn: 0.4556274	total: 75.5ms	remaining: 3

170:	learn: 0.4305049	total: 837ms	remaining: 4.06s
171:	learn: 0.4304763	total: 843ms	remaining: 4.06s
172:	learn: 0.4304112	total: 848ms	remaining: 4.05s
173:	learn: 0.4303534	total: 854ms	remaining: 4.05s
174:	learn: 0.4302696	total: 870ms	remaining: 4.1s
175:	learn: 0.4302252	total: 876ms	remaining: 4.1s
176:	learn: 0.4301808	total: 882ms	remaining: 4.1s
177:	learn: 0.4300727	total: 886ms	remaining: 4.09s
178:	learn: 0.4299012	total: 891ms	remaining: 4.09s
179:	learn: 0.4298362	total: 896ms	remaining: 4.08s
180:	learn: 0.4296536	total: 915ms	remaining: 4.14s
181:	learn: 0.4296039	total: 919ms	remaining: 4.13s
182:	learn: 0.4294512	total: 924ms	remaining: 4.13s
183:	learn: 0.4293665	total: 931ms	remaining: 4.13s
184:	learn: 0.4293208	total: 936ms	remaining: 4.12s
185:	learn: 0.4292576	total: 942ms	remaining: 4.12s
186:	learn: 0.4291487	total: 948ms	remaining: 4.12s
187:	learn: 0.4290837	total: 954ms	remaining: 4.12s
188:	learn: 0.4290320	total: 959ms	remaining: 4.11s
189:	learn: 0.4

349:	learn: 0.4171146	total: 1.82s	remaining: 3.38s
350:	learn: 0.4170993	total: 1.82s	remaining: 3.37s
351:	learn: 0.4170296	total: 1.83s	remaining: 3.37s
352:	learn: 0.4169507	total: 1.83s	remaining: 3.36s
353:	learn: 0.4168513	total: 1.84s	remaining: 3.36s
354:	learn: 0.4167911	total: 1.85s	remaining: 3.36s
355:	learn: 0.4167014	total: 1.85s	remaining: 3.35s
356:	learn: 0.4166459	total: 1.86s	remaining: 3.35s
357:	learn: 0.4165743	total: 1.86s	remaining: 3.34s
358:	learn: 0.4165271	total: 1.87s	remaining: 3.34s
359:	learn: 0.4164773	total: 1.87s	remaining: 3.33s
360:	learn: 0.4164068	total: 1.88s	remaining: 3.32s
361:	learn: 0.4163803	total: 1.88s	remaining: 3.31s
362:	learn: 0.4162936	total: 1.89s	remaining: 3.31s
363:	learn: 0.4162607	total: 1.89s	remaining: 3.3s
364:	learn: 0.4162378	total: 1.89s	remaining: 3.29s
365:	learn: 0.4161847	total: 1.9s	remaining: 3.29s
366:	learn: 0.4161019	total: 1.9s	remaining: 3.28s
367:	learn: 0.4160357	total: 1.9s	remaining: 3.27s
368:	learn: 0.41

525:	learn: 0.4074637	total: 2.65s	remaining: 2.39s
526:	learn: 0.4073959	total: 2.65s	remaining: 2.38s
527:	learn: 0.4073354	total: 2.66s	remaining: 2.38s
528:	learn: 0.4072885	total: 2.66s	remaining: 2.37s
529:	learn: 0.4072809	total: 2.67s	remaining: 2.36s
530:	learn: 0.4072246	total: 2.67s	remaining: 2.36s
531:	learn: 0.4072054	total: 2.67s	remaining: 2.35s
532:	learn: 0.4071934	total: 2.68s	remaining: 2.35s
533:	learn: 0.4071529	total: 2.68s	remaining: 2.34s
534:	learn: 0.4071255	total: 2.69s	remaining: 2.34s
535:	learn: 0.4070785	total: 2.69s	remaining: 2.33s
536:	learn: 0.4070224	total: 2.7s	remaining: 2.33s
537:	learn: 0.4069904	total: 2.71s	remaining: 2.32s
538:	learn: 0.4069657	total: 2.71s	remaining: 2.32s
539:	learn: 0.4069465	total: 2.71s	remaining: 2.31s
540:	learn: 0.4068997	total: 2.72s	remaining: 2.31s
541:	learn: 0.4068396	total: 2.72s	remaining: 2.3s
542:	learn: 0.4068345	total: 2.73s	remaining: 2.29s
543:	learn: 0.4067876	total: 2.73s	remaining: 2.29s
544:	learn: 0.

689:	learn: 0.3999873	total: 3.48s	remaining: 1.56s
690:	learn: 0.3999406	total: 3.48s	remaining: 1.56s
691:	learn: 0.3998636	total: 3.49s	remaining: 1.55s
692:	learn: 0.3998354	total: 3.49s	remaining: 1.55s
693:	learn: 0.3997625	total: 3.5s	remaining: 1.54s
694:	learn: 0.3997516	total: 3.5s	remaining: 1.54s
695:	learn: 0.3996609	total: 3.51s	remaining: 1.53s
696:	learn: 0.3996270	total: 3.51s	remaining: 1.53s
697:	learn: 0.3996092	total: 3.52s	remaining: 1.52s
698:	learn: 0.3995984	total: 3.52s	remaining: 1.52s
699:	learn: 0.3995530	total: 3.53s	remaining: 1.51s
700:	learn: 0.3995067	total: 3.53s	remaining: 1.51s
701:	learn: 0.3994952	total: 3.55s	remaining: 1.5s
702:	learn: 0.3994512	total: 3.56s	remaining: 1.5s
703:	learn: 0.3994372	total: 3.57s	remaining: 1.5s
704:	learn: 0.3994187	total: 3.57s	remaining: 1.49s
705:	learn: 0.3993986	total: 3.58s	remaining: 1.49s
706:	learn: 0.3993691	total: 3.58s	remaining: 1.48s
707:	learn: 0.3993149	total: 3.59s	remaining: 1.48s
708:	learn: 0.399

873:	learn: 0.3928733	total: 4.31s	remaining: 621ms
874:	learn: 0.3928520	total: 4.31s	remaining: 616ms
875:	learn: 0.3928432	total: 4.32s	remaining: 611ms
876:	learn: 0.3928421	total: 4.32s	remaining: 606ms
877:	learn: 0.3927852	total: 4.33s	remaining: 601ms
878:	learn: 0.3927685	total: 4.33s	remaining: 596ms
879:	learn: 0.3927371	total: 4.34s	remaining: 592ms
880:	learn: 0.3926994	total: 4.34s	remaining: 587ms
881:	learn: 0.3926419	total: 4.35s	remaining: 582ms
882:	learn: 0.3926159	total: 4.35s	remaining: 577ms
883:	learn: 0.3925730	total: 4.36s	remaining: 572ms
884:	learn: 0.3925382	total: 4.36s	remaining: 567ms
885:	learn: 0.3925160	total: 4.37s	remaining: 562ms
886:	learn: 0.3924578	total: 4.37s	remaining: 557ms
887:	learn: 0.3924438	total: 4.37s	remaining: 552ms
888:	learn: 0.3924399	total: 4.38s	remaining: 547ms
889:	learn: 0.3924084	total: 4.39s	remaining: 542ms
890:	learn: 0.3923838	total: 4.39s	remaining: 537ms
891:	learn: 0.3923512	total: 4.4s	remaining: 532ms
892:	learn: 0

In [18]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

preds = (preds >= 0.5).astype(int)
print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.442080
F1_score:  0.6238377007607777


In [21]:
model = CatBoostRegressor(iterations=1250, learning_rate=0.01,
                           depth = 7, random_seed=41)
model.fit(X_train, y_train)
preds = model.predict(X_test)

0:	learn: 0.4943248	total: 5.57ms	remaining: 6.95s
1:	learn: 0.4935052	total: 10.7ms	remaining: 6.71s
2:	learn: 0.4927762	total: 15.9ms	remaining: 6.6s
3:	learn: 0.4919559	total: 21.1ms	remaining: 6.56s
4:	learn: 0.4912490	total: 23.9ms	remaining: 5.96s
5:	learn: 0.4905749	total: 29ms	remaining: 6.01s
6:	learn: 0.4898118	total: 34.5ms	remaining: 6.13s
7:	learn: 0.4890648	total: 39.6ms	remaining: 6.15s
8:	learn: 0.4883679	total: 44.8ms	remaining: 6.18s
9:	learn: 0.4876758	total: 50ms	remaining: 6.21s
10:	learn: 0.4870893	total: 53.2ms	remaining: 6s
11:	learn: 0.4863941	total: 58.6ms	remaining: 6.05s
12:	learn: 0.4857690	total: 63.6ms	remaining: 6.05s
13:	learn: 0.4851549	total: 68.4ms	remaining: 6.04s
14:	learn: 0.4844420	total: 73.4ms	remaining: 6.04s
15:	learn: 0.4838115	total: 78.4ms	remaining: 6.05s
16:	learn: 0.4832128	total: 82.3ms	remaining: 5.97s
17:	learn: 0.4826030	total: 87.4ms	remaining: 5.98s
18:	learn: 0.4820027	total: 92.4ms	remaining: 5.99s
19:	learn: 0.4814333	total: 95

179:	learn: 0.4462751	total: 1.02s	remaining: 6.09s
180:	learn: 0.4462054	total: 1.03s	remaining: 6.08s
181:	learn: 0.4461300	total: 1.04s	remaining: 6.08s
182:	learn: 0.4460355	total: 1.04s	remaining: 6.08s
183:	learn: 0.4459500	total: 1.05s	remaining: 6.08s
184:	learn: 0.4458693	total: 1.05s	remaining: 6.08s
185:	learn: 0.4458317	total: 1.06s	remaining: 6.06s
186:	learn: 0.4457304	total: 1.07s	remaining: 6.06s
187:	learn: 0.4456423	total: 1.07s	remaining: 6.06s
188:	learn: 0.4455729	total: 1.08s	remaining: 6.07s
189:	learn: 0.4455004	total: 1.09s	remaining: 6.08s
190:	learn: 0.4454392	total: 1.1s	remaining: 6.12s
191:	learn: 0.4453692	total: 1.11s	remaining: 6.12s
192:	learn: 0.4453043	total: 1.12s	remaining: 6.13s
193:	learn: 0.4452481	total: 1.13s	remaining: 6.17s
194:	learn: 0.4452045	total: 1.15s	remaining: 6.22s
195:	learn: 0.4451807	total: 1.15s	remaining: 6.2s
196:	learn: 0.4451005	total: 1.16s	remaining: 6.19s
197:	learn: 0.4450081	total: 1.16s	remaining: 6.19s
198:	learn: 0.

339:	learn: 0.4386784	total: 2.04s	remaining: 5.47s
340:	learn: 0.4386508	total: 2.05s	remaining: 5.47s
341:	learn: 0.4386117	total: 2.06s	remaining: 5.46s
342:	learn: 0.4385699	total: 2.06s	remaining: 5.45s
343:	learn: 0.4385457	total: 2.07s	remaining: 5.45s
344:	learn: 0.4385118	total: 2.07s	remaining: 5.44s
345:	learn: 0.4384734	total: 2.08s	remaining: 5.44s
346:	learn: 0.4384280	total: 2.09s	remaining: 5.43s
347:	learn: 0.4383975	total: 2.09s	remaining: 5.43s
348:	learn: 0.4383765	total: 2.1s	remaining: 5.43s
349:	learn: 0.4383454	total: 2.11s	remaining: 5.42s
350:	learn: 0.4382773	total: 2.12s	remaining: 5.42s
351:	learn: 0.4382622	total: 2.12s	remaining: 5.42s
352:	learn: 0.4382272	total: 2.13s	remaining: 5.42s
353:	learn: 0.4382000	total: 2.15s	remaining: 5.43s
354:	learn: 0.4381882	total: 2.15s	remaining: 5.42s
355:	learn: 0.4381512	total: 2.16s	remaining: 5.42s
356:	learn: 0.4381154	total: 2.17s	remaining: 5.42s
357:	learn: 0.4380752	total: 2.17s	remaining: 5.41s
358:	learn: 0

516:	learn: 0.4339364	total: 3.06s	remaining: 4.34s
517:	learn: 0.4339080	total: 3.07s	remaining: 4.33s
518:	learn: 0.4339012	total: 3.07s	remaining: 4.33s
519:	learn: 0.4338910	total: 3.08s	remaining: 4.32s
520:	learn: 0.4338698	total: 3.09s	remaining: 4.32s
521:	learn: 0.4338372	total: 3.09s	remaining: 4.31s
522:	learn: 0.4338190	total: 3.1s	remaining: 4.31s
523:	learn: 0.4337963	total: 3.11s	remaining: 4.31s
524:	learn: 0.4337844	total: 3.11s	remaining: 4.3s
525:	learn: 0.4337574	total: 3.12s	remaining: 4.29s
526:	learn: 0.4337413	total: 3.13s	remaining: 4.29s
527:	learn: 0.4337348	total: 3.13s	remaining: 4.28s
528:	learn: 0.4337337	total: 3.13s	remaining: 4.27s
529:	learn: 0.4336995	total: 3.14s	remaining: 4.26s
530:	learn: 0.4336823	total: 3.14s	remaining: 4.26s
531:	learn: 0.4336766	total: 3.15s	remaining: 4.25s
532:	learn: 0.4336394	total: 3.15s	remaining: 4.24s
533:	learn: 0.4336114	total: 3.16s	remaining: 4.24s
534:	learn: 0.4335935	total: 3.16s	remaining: 4.23s
535:	learn: 0.

679:	learn: 0.4311049	total: 3.9s	remaining: 3.27s
680:	learn: 0.4310841	total: 3.91s	remaining: 3.27s
681:	learn: 0.4310760	total: 3.92s	remaining: 3.26s
682:	learn: 0.4310491	total: 3.92s	remaining: 3.26s
683:	learn: 0.4310155	total: 3.93s	remaining: 3.25s
684:	learn: 0.4309938	total: 3.93s	remaining: 3.25s
685:	learn: 0.4309780	total: 3.94s	remaining: 3.24s
686:	learn: 0.4309373	total: 3.94s	remaining: 3.23s
687:	learn: 0.4309258	total: 3.95s	remaining: 3.23s
688:	learn: 0.4309170	total: 3.96s	remaining: 3.22s
689:	learn: 0.4309060	total: 3.96s	remaining: 3.21s
690:	learn: 0.4308749	total: 3.97s	remaining: 3.21s
691:	learn: 0.4308652	total: 3.97s	remaining: 3.2s
692:	learn: 0.4308419	total: 3.98s	remaining: 3.19s
693:	learn: 0.4308321	total: 3.98s	remaining: 3.19s
694:	learn: 0.4308231	total: 3.98s	remaining: 3.18s
695:	learn: 0.4308094	total: 3.99s	remaining: 3.17s
696:	learn: 0.4307732	total: 4s	remaining: 3.17s
697:	learn: 0.4307420	total: 4s	remaining: 3.16s
698:	learn: 0.430738

838:	learn: 0.4281734	total: 4.75s	remaining: 2.33s
839:	learn: 0.4281529	total: 4.76s	remaining: 2.32s
840:	learn: 0.4281233	total: 4.76s	remaining: 2.31s
841:	learn: 0.4280863	total: 4.78s	remaining: 2.31s
842:	learn: 0.4280501	total: 4.78s	remaining: 2.31s
843:	learn: 0.4280175	total: 4.79s	remaining: 2.3s
844:	learn: 0.4280048	total: 4.79s	remaining: 2.3s
845:	learn: 0.4279911	total: 4.8s	remaining: 2.29s
846:	learn: 0.4279853	total: 4.8s	remaining: 2.28s
847:	learn: 0.4279474	total: 4.81s	remaining: 2.28s
848:	learn: 0.4279362	total: 4.81s	remaining: 2.27s
849:	learn: 0.4278829	total: 4.82s	remaining: 2.27s
850:	learn: 0.4278770	total: 4.82s	remaining: 2.26s
851:	learn: 0.4278642	total: 4.83s	remaining: 2.26s
852:	learn: 0.4278565	total: 4.83s	remaining: 2.25s
853:	learn: 0.4278461	total: 4.84s	remaining: 2.24s
854:	learn: 0.4278244	total: 4.84s	remaining: 2.24s
855:	learn: 0.4278108	total: 4.85s	remaining: 2.23s
856:	learn: 0.4277873	total: 4.86s	remaining: 2.23s
857:	learn: 0.42

1020:	learn: 0.4241030	total: 5.76s	remaining: 1.29s
1021:	learn: 0.4240883	total: 5.77s	remaining: 1.29s
1022:	learn: 0.4240757	total: 5.78s	remaining: 1.28s
1023:	learn: 0.4240488	total: 5.79s	remaining: 1.28s
1024:	learn: 0.4240371	total: 5.8s	remaining: 1.27s
1025:	learn: 0.4240248	total: 5.8s	remaining: 1.27s
1026:	learn: 0.4239943	total: 5.81s	remaining: 1.26s
1027:	learn: 0.4239777	total: 5.82s	remaining: 1.26s
1028:	learn: 0.4239619	total: 5.82s	remaining: 1.25s
1029:	learn: 0.4239515	total: 5.83s	remaining: 1.24s
1030:	learn: 0.4239241	total: 5.83s	remaining: 1.24s
1031:	learn: 0.4239098	total: 5.84s	remaining: 1.23s
1032:	learn: 0.4238958	total: 5.84s	remaining: 1.23s
1033:	learn: 0.4238479	total: 5.85s	remaining: 1.22s
1034:	learn: 0.4238246	total: 5.85s	remaining: 1.22s
1035:	learn: 0.4238060	total: 5.86s	remaining: 1.21s
1036:	learn: 0.4237831	total: 5.86s	remaining: 1.2s
1037:	learn: 0.4237720	total: 5.87s	remaining: 1.2s
1038:	learn: 0.4237457	total: 5.87s	remaining: 1.1

1206:	learn: 0.4202307	total: 6.79s	remaining: 242ms
1207:	learn: 0.4202110	total: 6.8s	remaining: 236ms
1208:	learn: 0.4201647	total: 6.81s	remaining: 231ms
1209:	learn: 0.4201466	total: 6.81s	remaining: 225ms
1210:	learn: 0.4201321	total: 6.82s	remaining: 220ms
1211:	learn: 0.4201056	total: 6.83s	remaining: 214ms
1212:	learn: 0.4200948	total: 6.83s	remaining: 208ms
1213:	learn: 0.4200868	total: 6.83s	remaining: 203ms
1214:	learn: 0.4200593	total: 6.84s	remaining: 197ms
1215:	learn: 0.4200349	total: 6.85s	remaining: 191ms
1216:	learn: 0.4200112	total: 6.85s	remaining: 186ms
1217:	learn: 0.4199977	total: 6.86s	remaining: 180ms
1218:	learn: 0.4199746	total: 6.86s	remaining: 174ms
1219:	learn: 0.4199598	total: 6.87s	remaining: 169ms
1220:	learn: 0.4199451	total: 6.87s	remaining: 163ms
1221:	learn: 0.4199364	total: 6.88s	remaining: 158ms
1222:	learn: 0.4199139	total: 6.88s	remaining: 152ms
1223:	learn: 0.4198826	total: 6.89s	remaining: 146ms
1224:	learn: 0.4198616	total: 6.89s	remaining: 

In [22]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

preds = (preds >= 0.5).astype(int)
print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.439847
F1_score:  0.6206308610400681


In [23]:
#Pruebo con CatBoostClassifier
#CatboostClassifier por defecto
model = CatBoostClassifier(random_seed = 41)
model.fit(X_train, y_train)
preds = model.predict(X_test)

Learning rate set to 0.022283
0:	learn: 0.6881339	total: 7.23ms	remaining: 7.22s
1:	learn: 0.6834910	total: 14.1ms	remaining: 7.05s
2:	learn: 0.6801623	total: 18.7ms	remaining: 6.21s
3:	learn: 0.6764096	total: 25.6ms	remaining: 6.37s
4:	learn: 0.6731570	total: 33.2ms	remaining: 6.6s
5:	learn: 0.6691407	total: 40.2ms	remaining: 6.66s
6:	learn: 0.6649513	total: 47.3ms	remaining: 6.71s
7:	learn: 0.6611457	total: 54.3ms	remaining: 6.73s
8:	learn: 0.6581740	total: 61.1ms	remaining: 6.73s
9:	learn: 0.6553907	total: 67.8ms	remaining: 6.71s
10:	learn: 0.6525744	total: 75ms	remaining: 6.75s
11:	learn: 0.6498927	total: 82.1ms	remaining: 6.76s
12:	learn: 0.6468371	total: 89.5ms	remaining: 6.79s
13:	learn: 0.6442128	total: 96.5ms	remaining: 6.79s
14:	learn: 0.6414336	total: 104ms	remaining: 6.82s
15:	learn: 0.6385736	total: 111ms	remaining: 6.82s
16:	learn: 0.6364580	total: 118ms	remaining: 6.81s
17:	learn: 0.6348948	total: 122ms	remaining: 6.68s
18:	learn: 0.6325626	total: 129ms	remaining: 6.69s


170:	learn: 0.5634857	total: 1.42s	remaining: 6.89s
171:	learn: 0.5634431	total: 1.43s	remaining: 6.88s
172:	learn: 0.5633482	total: 1.44s	remaining: 6.9s
173:	learn: 0.5632581	total: 1.45s	remaining: 6.89s
174:	learn: 0.5630650	total: 1.46s	remaining: 6.87s
175:	learn: 0.5629522	total: 1.47s	remaining: 6.86s
176:	learn: 0.5629389	total: 1.47s	remaining: 6.84s
177:	learn: 0.5627990	total: 1.48s	remaining: 6.82s
178:	learn: 0.5626435	total: 1.48s	remaining: 6.81s
179:	learn: 0.5625254	total: 1.49s	remaining: 6.79s
180:	learn: 0.5624150	total: 1.5s	remaining: 6.78s
181:	learn: 0.5623352	total: 1.51s	remaining: 6.77s
182:	learn: 0.5623251	total: 1.51s	remaining: 6.75s
183:	learn: 0.5622195	total: 1.52s	remaining: 6.73s
184:	learn: 0.5620687	total: 1.52s	remaining: 6.72s
185:	learn: 0.5619428	total: 1.53s	remaining: 6.71s
186:	learn: 0.5619063	total: 1.54s	remaining: 6.7s
187:	learn: 0.5618226	total: 1.55s	remaining: 6.68s
188:	learn: 0.5616596	total: 1.55s	remaining: 6.67s
189:	learn: 0.5

344:	learn: 0.5469547	total: 2.81s	remaining: 5.34s
345:	learn: 0.5469246	total: 2.82s	remaining: 5.33s
346:	learn: 0.5468397	total: 2.83s	remaining: 5.32s
347:	learn: 0.5468176	total: 2.84s	remaining: 5.32s
348:	learn: 0.5466689	total: 2.85s	remaining: 5.31s
349:	learn: 0.5465839	total: 2.85s	remaining: 5.3s
350:	learn: 0.5464869	total: 2.86s	remaining: 5.29s
351:	learn: 0.5464375	total: 2.87s	remaining: 5.28s
352:	learn: 0.5462495	total: 2.88s	remaining: 5.27s
353:	learn: 0.5460864	total: 2.88s	remaining: 5.26s
354:	learn: 0.5460625	total: 2.89s	remaining: 5.25s
355:	learn: 0.5459567	total: 2.9s	remaining: 5.25s
356:	learn: 0.5459216	total: 2.91s	remaining: 5.24s
357:	learn: 0.5458847	total: 2.91s	remaining: 5.22s
358:	learn: 0.5456888	total: 2.92s	remaining: 5.21s
359:	learn: 0.5455082	total: 2.93s	remaining: 5.2s
360:	learn: 0.5453953	total: 2.93s	remaining: 5.19s
361:	learn: 0.5453810	total: 2.94s	remaining: 5.18s
362:	learn: 0.5452671	total: 2.95s	remaining: 5.18s
363:	learn: 0.5

520:	learn: 0.5281174	total: 4.21s	remaining: 3.87s
521:	learn: 0.5280568	total: 4.22s	remaining: 3.87s
522:	learn: 0.5279885	total: 4.23s	remaining: 3.86s
523:	learn: 0.5278543	total: 4.24s	remaining: 3.85s
524:	learn: 0.5277332	total: 4.24s	remaining: 3.84s
525:	learn: 0.5276662	total: 4.25s	remaining: 3.83s
526:	learn: 0.5275206	total: 4.26s	remaining: 3.82s
527:	learn: 0.5274631	total: 4.26s	remaining: 3.81s
528:	learn: 0.5273855	total: 4.27s	remaining: 3.8s
529:	learn: 0.5272739	total: 4.28s	remaining: 3.79s
530:	learn: 0.5271160	total: 4.29s	remaining: 3.79s
531:	learn: 0.5269841	total: 4.29s	remaining: 3.78s
532:	learn: 0.5269179	total: 4.3s	remaining: 3.77s
533:	learn: 0.5268572	total: 4.31s	remaining: 3.76s
534:	learn: 0.5267936	total: 4.31s	remaining: 3.75s
535:	learn: 0.5267490	total: 4.32s	remaining: 3.74s
536:	learn: 0.5266866	total: 4.33s	remaining: 3.73s
537:	learn: 0.5266071	total: 4.33s	remaining: 3.72s
538:	learn: 0.5264727	total: 4.34s	remaining: 3.71s
539:	learn: 0.

683:	learn: 0.5140705	total: 5.44s	remaining: 2.51s
684:	learn: 0.5139770	total: 5.45s	remaining: 2.5s
685:	learn: 0.5139120	total: 5.46s	remaining: 2.5s
686:	learn: 0.5137979	total: 5.47s	remaining: 2.49s
687:	learn: 0.5137168	total: 5.48s	remaining: 2.48s
688:	learn: 0.5135773	total: 5.49s	remaining: 2.48s
689:	learn: 0.5134671	total: 5.5s	remaining: 2.47s
690:	learn: 0.5134012	total: 5.51s	remaining: 2.46s
691:	learn: 0.5133644	total: 5.51s	remaining: 2.45s
692:	learn: 0.5132651	total: 5.52s	remaining: 2.44s
693:	learn: 0.5132092	total: 5.53s	remaining: 2.44s
694:	learn: 0.5131051	total: 5.53s	remaining: 2.43s
695:	learn: 0.5130579	total: 5.54s	remaining: 2.42s
696:	learn: 0.5129235	total: 5.55s	remaining: 2.41s
697:	learn: 0.5127950	total: 5.56s	remaining: 2.4s
698:	learn: 0.5126680	total: 5.56s	remaining: 2.4s
699:	learn: 0.5126175	total: 5.57s	remaining: 2.39s
700:	learn: 0.5125165	total: 5.58s	remaining: 2.38s
701:	learn: 0.5124897	total: 5.59s	remaining: 2.37s
702:	learn: 0.512

861:	learn: 0.5004659	total: 6.84s	remaining: 1.09s
862:	learn: 0.5003716	total: 6.85s	remaining: 1.09s
863:	learn: 0.5003121	total: 6.86s	remaining: 1.08s
864:	learn: 0.5002637	total: 6.87s	remaining: 1.07s
865:	learn: 0.5001435	total: 6.88s	remaining: 1.06s
866:	learn: 0.5001253	total: 6.89s	remaining: 1.06s
867:	learn: 0.4999976	total: 6.9s	remaining: 1.05s
868:	learn: 0.4999080	total: 6.91s	remaining: 1.04s
869:	learn: 0.4998473	total: 6.92s	remaining: 1.03s
870:	learn: 0.4998211	total: 6.93s	remaining: 1.02s
871:	learn: 0.4997050	total: 6.93s	remaining: 1.02s
872:	learn: 0.4996308	total: 6.94s	remaining: 1.01s
873:	learn: 0.4995683	total: 6.95s	remaining: 1s
874:	learn: 0.4995006	total: 6.96s	remaining: 994ms
875:	learn: 0.4994051	total: 6.97s	remaining: 986ms
876:	learn: 0.4993305	total: 6.97s	remaining: 978ms
877:	learn: 0.4992557	total: 6.98s	remaining: 970ms
878:	learn: 0.4992006	total: 6.99s	remaining: 962ms
879:	learn: 0.4991746	total: 7s	remaining: 954ms
880:	learn: 0.49904

In [24]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.543571
F1_score:  0.6173469387755102


In [25]:
model = CatBoostClassifier(iterations = 1250, learning_rate = 0.01,
                           depth = 7, random_seed = 41)
model.fit(X_train, y_train)
preds = model.predict(X_test)

0:	learn: 0.6908385	total: 8.39ms	remaining: 10.5s
1:	learn: 0.6888662	total: 16.4ms	remaining: 10.3s
2:	learn: 0.6866983	total: 24.6ms	remaining: 10.2s
3:	learn: 0.6848295	total: 32.2ms	remaining: 10s
4:	learn: 0.6832139	total: 37.2ms	remaining: 9.26s
5:	learn: 0.6811713	total: 45.3ms	remaining: 9.38s
6:	learn: 0.6793942	total: 53.5ms	remaining: 9.51s
7:	learn: 0.6771978	total: 62ms	remaining: 9.63s
8:	learn: 0.6751186	total: 70.1ms	remaining: 9.67s
9:	learn: 0.6735083	total: 77.1ms	remaining: 9.56s
10:	learn: 0.6719591	total: 85.4ms	remaining: 9.61s
11:	learn: 0.6699953	total: 93.5ms	remaining: 9.65s
12:	learn: 0.6683183	total: 99.8ms	remaining: 9.49s
13:	learn: 0.6665101	total: 108ms	remaining: 9.53s
14:	learn: 0.6645567	total: 116ms	remaining: 9.56s
15:	learn: 0.6630689	total: 124ms	remaining: 9.58s
16:	learn: 0.6616089	total: 133ms	remaining: 9.64s
17:	learn: 0.6598867	total: 142ms	remaining: 9.69s
18:	learn: 0.6584803	total: 151ms	remaining: 9.81s
19:	learn: 0.6570482	total: 160m

178:	learn: 0.5769262	total: 1.58s	remaining: 9.46s
179:	learn: 0.5767559	total: 1.6s	remaining: 9.5s
180:	learn: 0.5766298	total: 1.62s	remaining: 9.54s
181:	learn: 0.5764332	total: 1.63s	remaining: 9.54s
182:	learn: 0.5763153	total: 1.63s	remaining: 9.53s
183:	learn: 0.5761546	total: 1.64s	remaining: 9.52s
184:	learn: 0.5759412	total: 1.65s	remaining: 9.51s
185:	learn: 0.5757053	total: 1.66s	remaining: 9.5s
186:	learn: 0.5755655	total: 1.67s	remaining: 9.49s
187:	learn: 0.5753316	total: 1.68s	remaining: 9.48s
188:	learn: 0.5751852	total: 1.69s	remaining: 9.46s
189:	learn: 0.5750238	total: 1.69s	remaining: 9.45s
190:	learn: 0.5748807	total: 1.7s	remaining: 9.44s
191:	learn: 0.5746431	total: 1.71s	remaining: 9.42s
192:	learn: 0.5744965	total: 1.72s	remaining: 9.41s
193:	learn: 0.5744283	total: 1.72s	remaining: 9.38s
194:	learn: 0.5742585	total: 1.73s	remaining: 9.37s
195:	learn: 0.5741167	total: 1.74s	remaining: 9.36s
196:	learn: 0.5739761	total: 1.75s	remaining: 9.35s
197:	learn: 0.57

337:	learn: 0.5591333	total: 2.99s	remaining: 8.07s
338:	learn: 0.5590529	total: 3s	remaining: 8.06s
339:	learn: 0.5589065	total: 3.01s	remaining: 8.06s
340:	learn: 0.5588562	total: 3.02s	remaining: 8.05s
341:	learn: 0.5587463	total: 3.03s	remaining: 8.05s
342:	learn: 0.5586797	total: 3.04s	remaining: 8.04s
343:	learn: 0.5586255	total: 3.05s	remaining: 8.04s
344:	learn: 0.5585566	total: 3.06s	remaining: 8.03s
345:	learn: 0.5585044	total: 3.07s	remaining: 8.03s
346:	learn: 0.5584354	total: 3.08s	remaining: 8.02s
347:	learn: 0.5582485	total: 3.09s	remaining: 8.02s
348:	learn: 0.5582011	total: 3.1s	remaining: 8.01s
349:	learn: 0.5580888	total: 3.11s	remaining: 8.01s
350:	learn: 0.5580295	total: 3.12s	remaining: 8s
351:	learn: 0.5579240	total: 3.13s	remaining: 8s
352:	learn: 0.5577856	total: 3.15s	remaining: 8s
353:	learn: 0.5577285	total: 3.16s	remaining: 7.99s
354:	learn: 0.5576409	total: 3.17s	remaining: 7.99s
355:	learn: 0.5575673	total: 3.18s	remaining: 7.99s
356:	learn: 0.5574673	tot

512:	learn: 0.5478194	total: 4.59s	remaining: 6.6s
513:	learn: 0.5477533	total: 4.61s	remaining: 6.59s
514:	learn: 0.5476977	total: 4.62s	remaining: 6.6s
515:	learn: 0.5476480	total: 4.63s	remaining: 6.59s
516:	learn: 0.5475749	total: 4.64s	remaining: 6.58s
517:	learn: 0.5475433	total: 4.65s	remaining: 6.57s
518:	learn: 0.5475311	total: 4.66s	remaining: 6.56s
519:	learn: 0.5475139	total: 4.66s	remaining: 6.55s
520:	learn: 0.5474146	total: 4.67s	remaining: 6.54s
521:	learn: 0.5473041	total: 4.68s	remaining: 6.53s
522:	learn: 0.5472927	total: 4.69s	remaining: 6.51s
523:	learn: 0.5472417	total: 4.7s	remaining: 6.5s
524:	learn: 0.5471908	total: 4.7s	remaining: 6.5s
525:	learn: 0.5471460	total: 4.71s	remaining: 6.49s
526:	learn: 0.5471081	total: 4.72s	remaining: 6.48s
527:	learn: 0.5470609	total: 4.73s	remaining: 6.47s
528:	learn: 0.5470253	total: 4.74s	remaining: 6.46s
529:	learn: 0.5469607	total: 4.75s	remaining: 6.45s
530:	learn: 0.5469288	total: 4.75s	remaining: 6.44s
531:	learn: 0.5468

678:	learn: 0.5402256	total: 6s	remaining: 5.05s
679:	learn: 0.5401547	total: 6.01s	remaining: 5.04s
680:	learn: 0.5401017	total: 6.03s	remaining: 5.04s
681:	learn: 0.5400367	total: 6.04s	remaining: 5.03s
682:	learn: 0.5399909	total: 6.04s	remaining: 5.02s
683:	learn: 0.5399490	total: 6.05s	remaining: 5.01s
684:	learn: 0.5398687	total: 6.06s	remaining: 5s
685:	learn: 0.5398187	total: 6.07s	remaining: 4.99s
686:	learn: 0.5397980	total: 6.08s	remaining: 4.98s
687:	learn: 0.5397696	total: 6.08s	remaining: 4.97s
688:	learn: 0.5396987	total: 6.09s	remaining: 4.96s
689:	learn: 0.5396290	total: 6.1s	remaining: 4.95s
690:	learn: 0.5395890	total: 6.11s	remaining: 4.94s
691:	learn: 0.5395772	total: 6.12s	remaining: 4.93s
692:	learn: 0.5395739	total: 6.13s	remaining: 4.92s
693:	learn: 0.5395182	total: 6.13s	remaining: 4.91s
694:	learn: 0.5394949	total: 6.14s	remaining: 4.91s
695:	learn: 0.5394556	total: 6.15s	remaining: 4.9s
696:	learn: 0.5394550	total: 6.16s	remaining: 4.88s
697:	learn: 0.539410

846:	learn: 0.5320066	total: 7.61s	remaining: 3.62s
847:	learn: 0.5319623	total: 7.62s	remaining: 3.61s
848:	learn: 0.5319206	total: 7.63s	remaining: 3.6s
849:	learn: 0.5318712	total: 7.64s	remaining: 3.6s
850:	learn: 0.5318282	total: 7.66s	remaining: 3.59s
851:	learn: 0.5317645	total: 7.67s	remaining: 3.58s
852:	learn: 0.5316508	total: 7.68s	remaining: 3.57s
853:	learn: 0.5315912	total: 7.7s	remaining: 3.57s
854:	learn: 0.5315387	total: 7.71s	remaining: 3.56s
855:	learn: 0.5314962	total: 7.72s	remaining: 3.55s
856:	learn: 0.5314421	total: 7.72s	remaining: 3.54s
857:	learn: 0.5313803	total: 7.73s	remaining: 3.53s
858:	learn: 0.5313341	total: 7.74s	remaining: 3.52s
859:	learn: 0.5312929	total: 7.75s	remaining: 3.51s
860:	learn: 0.5312469	total: 7.76s	remaining: 3.5s
861:	learn: 0.5311901	total: 7.76s	remaining: 3.5s
862:	learn: 0.5311548	total: 7.77s	remaining: 3.48s
863:	learn: 0.5310533	total: 7.78s	remaining: 3.48s
864:	learn: 0.5310232	total: 7.79s	remaining: 3.47s
865:	learn: 0.530

1018:	learn: 0.5214990	total: 9.19s	remaining: 2.08s
1019:	learn: 0.5213983	total: 9.2s	remaining: 2.07s
1020:	learn: 0.5213269	total: 9.21s	remaining: 2.07s
1021:	learn: 0.5212852	total: 9.22s	remaining: 2.06s
1022:	learn: 0.5211973	total: 9.23s	remaining: 2.05s
1023:	learn: 0.5211778	total: 9.24s	remaining: 2.04s
1024:	learn: 0.5211254	total: 9.24s	remaining: 2.03s
1025:	learn: 0.5210478	total: 9.25s	remaining: 2.02s
1026:	learn: 0.5210155	total: 9.26s	remaining: 2.01s
1027:	learn: 0.5209657	total: 9.27s	remaining: 2s
1028:	learn: 0.5208943	total: 9.28s	remaining: 1.99s
1029:	learn: 0.5208026	total: 9.29s	remaining: 1.98s
1030:	learn: 0.5207677	total: 9.29s	remaining: 1.97s
1031:	learn: 0.5207073	total: 9.3s	remaining: 1.97s
1032:	learn: 0.5206582	total: 9.31s	remaining: 1.96s
1033:	learn: 0.5205900	total: 9.32s	remaining: 1.95s
1034:	learn: 0.5205189	total: 9.33s	remaining: 1.94s
1035:	learn: 0.5204520	total: 9.34s	remaining: 1.93s
1036:	learn: 0.5203640	total: 9.34s	remaining: 1.92

1188:	learn: 0.5117167	total: 10.8s	remaining: 553ms
1189:	learn: 0.5116779	total: 10.8s	remaining: 544ms
1190:	learn: 0.5116336	total: 10.8s	remaining: 535ms
1191:	learn: 0.5115941	total: 10.8s	remaining: 526ms
1192:	learn: 0.5115262	total: 10.8s	remaining: 517ms
1193:	learn: 0.5114854	total: 10.8s	remaining: 508ms
1194:	learn: 0.5114374	total: 10.8s	remaining: 499ms
1195:	learn: 0.5113832	total: 10.8s	remaining: 490ms
1196:	learn: 0.5113391	total: 10.9s	remaining: 481ms
1197:	learn: 0.5113178	total: 10.9s	remaining: 472ms
1198:	learn: 0.5112768	total: 10.9s	remaining: 463ms
1199:	learn: 0.5112036	total: 10.9s	remaining: 454ms
1200:	learn: 0.5111365	total: 10.9s	remaining: 444ms
1201:	learn: 0.5110835	total: 10.9s	remaining: 435ms
1202:	learn: 0.5110215	total: 10.9s	remaining: 426ms
1203:	learn: 0.5109745	total: 10.9s	remaining: 417ms
1204:	learn: 0.5109246	total: 10.9s	remaining: 408ms
1205:	learn: 0.5108977	total: 10.9s	remaining: 399ms
1206:	learn: 0.5108426	total: 11s	remaining: 3

In [26]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

print("F1_score: ", f1_score(y_test, preds))

RMSE: 0.534435
F1_score:  0.6285226302305722
